In [133]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from dfply import *
from more_dfply import ifelse, case_when
from more_dfply.facets import text_facet, text_filter

In [134]:
circuits = pd.read_csv('./data/circuits.csv', na_values=r'\N')
constructor_results = pd.read_csv('./data/constructor_results.csv')
constructor_standings = pd.read_csv('./data/constructor_standings.csv')
constructors = pd.read_csv('data/constructors.csv')
driver_standings = pd.read_csv('data/driver_standings.csv')
drivers = pd.read_csv('data/drivers.csv', na_values=r'\N')
lap_times = pd.read_csv('data/lap_times.csv', na_values=r'\N')
pit_stops = pd.read_csv('data/pit_stops.csv', na_values=r'\N')
qualifying = pd.read_csv('data/qualifying.csv', na_values=r'\N')
races = pd.read_csv('data/races.csv', na_values = r'\N')
results = pd.read_csv('data/results.csv', na_values=r'\N')
seasons = pd.read_csv('data/seasons.csv')
sprint_results = pd.read_csv('data/sprint_results.csv')
status = pd.read_csv('data/status.csv')

In [135]:
constructor_to_standings = (constructors >> left_join(constructor_standings, by='constructorId'))
standings_to_results = (constructor_to_standings >> left_join(constructor_results, by=['constructorId', 'raceId']))
results_to_races = (standings_to_results >> left_join(races, by='raceId'))
results_to_races

,constructorId,constructorRef,name_x,nationality,url_x,constructorStandingsId,raceId,points_x,position,positionText,...,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,1.0,18.0,14.0,1.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,7.0,19.0,24.0,1.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,18.0,20.0,28.0,3.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,29.0,21.0,34.0,3.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,40.0,22.0,42.0,3.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,51.0,23.0,53.0,2.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,62.0,24.0,53.0,3.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,73.0,25.0,58.0,3.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,84.0,26.0,72.0,3.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [136]:
results_to_races.columns

Index(['constructorId', 'constructorRef', 'name_x', 'nationality', 'url_x',
       'constructorStandingsId', 'raceId', 'points_x', 'position',
       'positionText', 'wins', 'constructorResultsId', 'points_y', 'status',
       'year', 'round', 'circuitId', 'name_y', 'date', 'time', 'url_y',
       'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time',
       'quali_date', 'quali_time', 'sprint_date', 'sprint_time'],
      dtype='object')

In [137]:
split_and_get = lambda character, position: X.date.str.split(character).str.get(position)
qualified = lambda teamname, year: if_else((X['name'] == teamname) & (X.year == year), "yes", "no")

results_cleaned = (results_to_races
                    >> select(X.constructorId, X.constructorRef, X.name_x, X.nationality, X.constructorStandingsId, X.raceId, X.points_x, X.year, X.circuitId, X.date)
                    >> mutate(name = X.name_x)
                    >> mutate(points = X.points_x)
                    >> drop(X.points_x, X.name_x, X.year) ### dropped year because the dtype was float as well as the date column provides exact date, using that to grab info
                    >> group_by(X.name)
                    >> mutate(year = split_and_get('-', 0))
                    >> mutate(month = split_and_get('-', 1))
                    >> mutate(day = split_and_get('-', 2))
                    >> drop(X.date)
                    # >> mutate(era = ifelse(X.year >= 1950 & X.year <= 1951, 'Front Engine Era', 'other'))
                    >> mutate(disqualified = qualified('McLaren', '2007'))
                    >> filter_by(X.disqualified == 'no')
)
results_cleaned

,constructorId,constructorRef,nationality,constructorStandingsId,raceId,circuitId,name,points,year,month,day,disqualified
11375,147,afm,German,NaN,NaN,NaN,AFM,NaN,NaN,NaN,NaN,no
8372,39,ags,French,8254.0,306.0,18.0,AGS,0.0,1991,03,24,no
8373,39,ags,French,8272.0,307.0,21.0,AGS,0.0,1991,04,28,no
8374,39,ags,French,8291.0,308.0,6.0,AGS,0.0,1991,05,12,no
8375,39,ags,French,8310.0,309.0,7.0,AGS,0.0,1991,06,02,no
8376,39,ags,French,8329.0,310.0,32.0,AGS,0.0,1991,06,16,no
8377,39,ags,French,8348.0,311.0,8.0,AGS,0.0,1991,07,07,no
8378,39,ags,French,8367.0,312.0,9.0,AGS,0.0,1991,07,14,no
8379,39,ags,French,8386.0,313.0,10.0,AGS,0.0,1991,07,28,no
...,...,...,...,...,...,...,...,...,...,...,...,...


In [138]:
results_cleaned.dtypes

constructorId               int64
constructorRef             object
nationality                object
constructorStandingsId    float64
raceId                    float64
circuitId                 float64
name                       object
points                    float64
year                       object
month                      object
day                        object
disqualified               object
dtype: object

In [139]:
seasons_points = (results_cleaned
                    >> group_by(X.year, X.name, X.constructorId, X.constructorRef, X.nationality)
                    >> summarize(finalpoints = X.points.max())
)
seasons_points

,nationality,constructorRef,constructorId,name,year,finalpoints
0,British,brm,66,BRM,1958,18.0
1,British,connaught,125,Connaught,1958,0.0
2,British,cooper,87,Cooper,1958,31.0
3,Italian,ferrari,6,Ferrari,1958,40.0
4,Italian,maserati,105,Maserati,1958,6.0
5,Italian,osca,127,OSCA,1958,0.0
6,German,porsche,95,Porsche,1958,0.0
7,British,team_lotus,32,Team Lotus,1958,3.0
8,British,vanwall,118,Vanwall,1958,48.0
...,...,...,...,...,...,...


In [140]:
truemax = (seasons_points
            >> group_by(X.year)
            >> summarize(maxpoints = X.finalpoints.max()))
truemax

,year,maxpoints
0,1958,48.0
1,1959,40.0
2,1960,48.0
3,1961,45.0
4,1962,42.0
5,1963,54.0
6,1964,45.0
7,1965,54.0
8,1966,42.0
...,...,...


In [141]:
pd.set_option('display.max_rows', False)

names_recode = {'Team Lotus':'Lotus-Ford', 'Tyrrell':'Tyrrell-Ford', 'Williams':'Williams-Ford', 'Ferrari':'Ferrari', 'Cooper-Climax':'Cooper-Climax',
                'BRM':'BRM', 'Lotus-Climax':'Lotus-Climax', 'Brabham-Repco':'Brabham-Repco', 'Lotus-Ford':'Lotus-Ford', 'Matra-Ford':'Matra-Ford',
                'McLaren':'McLaren', 'Benetton':'Benetton', 'Renault':'Renault', 'Brawn':'Brawn', 'Red Bull':'Red Bull', 'Mercedes':'Mercedes', 'Vanwall':'Vanwall'}


points_max_join = (seasons_points >> inner_join(truemax,by='year'))
constructor_champions = (points_max_join
             >> mutate(team_percents = X.finalpoints/X.maxpoints)
             >> mutate(name = X.name.map(names_recode))
             >> filter_by(X.team_percents==1)
             >> drop(X.team_percents, X.maxpoints)
             >> mutate(constructorId = X.constructorId.astype(str))
)
constructor_champions

,nationality,constructorRef,constructorId,name,year,finalpoints
8,British,vanwall,118,Vanwall,1958,48.0
13,British,cooper-climax,170,Cooper-Climax,1959,40.0
29,British,cooper-climax,170,Cooper-Climax,1960,48.0
44,Italian,ferrari,6,Ferrari,1961,45.0
51,British,brm,66,BRM,1962,42.0
85,British,lotus-climax,172,Lotus-Climax,1963,54.0
99,Italian,ferrari,6,Ferrari,1964,45.0
118,British,lotus-climax,172,Lotus-Climax,1965,54.0
124,British,brabham-repco,191,Brabham-Repco,1966,42.0
...,...,...,...,...,...,...


***years off*** - 
***2007(mclaren -> ferrari) because mclaren was disqualfied that season***

In [142]:
countedteams = (constructor_champions
                >> group_by(X.name)
                >> summarize(championships = X.constructorRef.count())
                >> ungroup
                >> arrange(X.championships, ascending = False)
)
countedteams

,name,championships
5,Ferrari,16
15,Williams-Ford,9
9,McLaren,8
10,Mercedes,8
7,Lotus-Ford,5
11,Red Bull,5
2,Brabham-Repco,2
4,Cooper-Climax,2
6,Lotus-Climax,2
12,Renault,2


In [143]:
teamsoverfive = (countedteams
            >> filter_by(X.championships >= 5)
)
teamsoverfive

,name,championships
5,Ferrari,16
15,Williams-Ford,9
9,McLaren,8
10,Mercedes,8
7,Lotus-Ford,5
11,Red Bull,5


In [144]:
topteams = (points_max_join
            >> mutate(team_percents = X.finalpoints/65)
            >> mutate(name = X.name.map(names_recode))
            >> group_by(X.name)
            >> summarize(overall = X.team_percents.mean())
            >> ungroup
            >> arrange(X.overall, ascending = False)
)
topteams

,name,overall
10,Mercedes,7.813609
11,Red Bull,5.179487
3,Brawn,2.646154
5,Ferrari,2.131124
9,McLaren,1.715237
15,Williams-Ford,1.176105
12,Renault,1.139103
8,Matra-Ford,0.853846
1,Benetton,0.828365
6,Lotus-Climax,0.492308


In [145]:
# Why are the years off?

find_2007 = (seasons_points
            >> group_by (X.name)
            >> filter_by(X.year == '2007')
)
find_2007

,nationality,constructorRef,constructorId,name,year,finalpoints
729,German,bmw_sauber,2,BMW Sauber,2007,101.0
730,Italian,ferrari,6,Ferrari,2007,204.0
731,Japanese,honda,11,Honda,2007,6.0
732,Austrian,red_bull,9,Red Bull,2007,24.0
733,French,renault,4,Renault,2007,51.0
734,Dutch,spyker,12,Spyker,2007,1.0
735,Japanese,super_aguri,8,Super Aguri,2007,4.0
736,Italian,toro_rosso,5,Toro Rosso,2007,8.0
737,Japanese,toyota,7,Toyota,2007,13.0
738,British,williams,3,Williams,2007,33.0


> **Which Driver is the Best over the Eras of F1**

In [146]:
drivers_joined = (drivers >> left_join(driver_standings, by='driverId'))
drivers_joined.head(10)

,driverId,driverRef,number,code,forename,surname,dob,nationality,url,driverStandingsId,raceId,points,position,positionText,wins
0,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,1.0,18.0,10.0,1.0,1,1.0
1,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,9.0,19.0,14.0,1.0,1,1.0
2,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,27.0,20.0,14.0,3.0,3,1.0
3,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,48.0,21.0,20.0,2.0,2,1.0
4,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,69.0,22.0,28.0,3.0,3,1.0
5,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,91.0,23.0,38.0,1.0,1,2.0
6,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,113.0,24.0,38.0,2.0,2,2.0
7,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,135.0,25.0,38.0,4.0,4,2.0
8,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,157.0,26.0,48.0,1.0,1,3.0
9,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,179.0,27.0,58.0,1.0,1,4.0


In [147]:
drivers_cleaned = (drivers_joined
                    >> drop(X.dob, X.url, X.driverRef)
)
drivers_cleaned.head(8)

,driverId,number,code,forename,surname,nationality,driverStandingsId,raceId,points,position,positionText,wins
0,1,44.0,HAM,Lewis,Hamilton,British,1.0,18.0,10.0,1.0,1,1.0
1,1,44.0,HAM,Lewis,Hamilton,British,9.0,19.0,14.0,1.0,1,1.0
2,1,44.0,HAM,Lewis,Hamilton,British,27.0,20.0,14.0,3.0,3,1.0
3,1,44.0,HAM,Lewis,Hamilton,British,48.0,21.0,20.0,2.0,2,1.0
4,1,44.0,HAM,Lewis,Hamilton,British,69.0,22.0,28.0,3.0,3,1.0
5,1,44.0,HAM,Lewis,Hamilton,British,91.0,23.0,38.0,1.0,1,2.0
6,1,44.0,HAM,Lewis,Hamilton,British,113.0,24.0,38.0,2.0,2,2.0
7,1,44.0,HAM,Lewis,Hamilton,British,135.0,25.0,38.0,4.0,4,2.0


In [148]:
drivers_races = (drivers_cleaned >> left_join(races, by='raceId'))
drivers_races.columns

Index(['driverId', 'number', 'code', 'forename', 'surname', 'nationality',
       'driverStandingsId', 'raceId', 'points', 'position', 'positionText',
       'wins', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url',
       'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time',
       'quali_date', 'quali_time', 'sprint_date', 'sprint_time'],
      dtype='object')

In [149]:
drivers_cleaned2 = (drivers_races
                    >> drop(X.fp1_time, X.fp1_date, X.fp2_date, X.fp2_time, X.fp3_date, X.fp3_time, X.quali_date, X.quali_time, X.url, 
                            X.sprint_date, X.sprint_time)
                    >> mutate(year = split_and_get('-', 0))
                    >> drop(X.date, X.number)
                    >> group_by(X.driverId, X.raceId)
)
drivers_cleaned2.head(5)

,driverId,code,forename,surname,nationality,driverStandingsId,raceId,points,position,positionText,wins,year,round,circuitId,name,time
0,1,HAM,Lewis,Hamilton,British,1.0,18.0,10.0,1.0,1,1.0,2008,1.0,1.0,Australian Grand Prix,04:30:00
1,1,HAM,Lewis,Hamilton,British,9.0,19.0,14.0,1.0,1,1.0,2008,2.0,2.0,Malaysian Grand Prix,07:00:00
2,1,HAM,Lewis,Hamilton,British,27.0,20.0,14.0,3.0,3,1.0,2008,3.0,3.0,Bahrain Grand Prix,11:30:00
3,1,HAM,Lewis,Hamilton,British,48.0,21.0,20.0,2.0,2,1.0,2008,4.0,4.0,Spanish Grand Prix,12:00:00
4,1,HAM,Lewis,Hamilton,British,69.0,22.0,28.0,3.0,3,1.0,2008,5.0,5.0,Turkish Grand Prix,12:00:00


In [150]:
drivers_results = (drivers_cleaned2 >> inner_join(results, by=['raceId', 'driverId']))
drivers_results.head(8)

,driverId,code,forename,surname,nationality,driverStandingsId,raceId,points_x,position_x,positionText_x,...,positionOrder,points_y,laps,time_y,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,HAM,Lewis,Hamilton,British,1.0,18.0,10.0,1.0,1,...,1,10.0,58,1:34:50.616,5690616.0,39.0,2.0,1:27.452,218.300,1
1,1,HAM,Lewis,Hamilton,British,9.0,19.0,14.0,1.0,1,...,5,4.0,56,+46.548,5525103.0,53.0,3.0,1:35.462,209.033,1
2,1,HAM,Lewis,Hamilton,British,27.0,20.0,14.0,3.0,3,...,13,0.0,56,NaN,NaN,25.0,19.0,1:35.520,203.969,11
3,1,HAM,Lewis,Hamilton,British,48.0,21.0,20.0,2.0,2,...,3,6.0,66,+4.187,5903238.0,20.0,3.0,1:22.017,204.323,1
4,1,HAM,Lewis,Hamilton,British,69.0,22.0,28.0,3.0,3,...,2,8.0,58,+3.779,5213230.0,31.0,2.0,1:26.529,222.085,1
5,1,HAM,Lewis,Hamilton,British,91.0,23.0,38.0,1.0,1,...,1,10.0,76,2:00:42.742,7242742.0,71.0,6.0,1:18.510,153.152,1
6,1,HAM,Lewis,Hamilton,British,113.0,24.0,38.0,2.0,2,...,19,0.0,19,NaN,NaN,4.0,3.0,1:17.506,202.559,4
7,1,HAM,Lewis,Hamilton,British,135.0,25.0,38.0,4.0,4,...,10,0.0,70,+54.538,5564783.0,40.0,5.0,1:17.453,205.022,1


In [151]:
drivers_results_cols = (drivers_results.rename(columns={"points_x":"running_season_total", "points_y":"race_points", "time_x":"time_of_race", 
                        "time_y":"race_time", "name":"circuit_name", "positionText_x":"season_position", 
                        "positionText_y":"race_position", "forename":"fname", "surname":"lname", "round":"race_round"})
                        )
drivers_results_cols.head(8)

,driverId,code,fname,lname,nationality,driverStandingsId,raceId,running_season_total,position_x,season_position,...,positionOrder,race_points,laps,race_time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,HAM,Lewis,Hamilton,British,1.0,18.0,10.0,1.0,1,...,1,10.0,58,1:34:50.616,5690616.0,39.0,2.0,1:27.452,218.300,1
1,1,HAM,Lewis,Hamilton,British,9.0,19.0,14.0,1.0,1,...,5,4.0,56,+46.548,5525103.0,53.0,3.0,1:35.462,209.033,1
2,1,HAM,Lewis,Hamilton,British,27.0,20.0,14.0,3.0,3,...,13,0.0,56,NaN,NaN,25.0,19.0,1:35.520,203.969,11
3,1,HAM,Lewis,Hamilton,British,48.0,21.0,20.0,2.0,2,...,3,6.0,66,+4.187,5903238.0,20.0,3.0,1:22.017,204.323,1
4,1,HAM,Lewis,Hamilton,British,69.0,22.0,28.0,3.0,3,...,2,8.0,58,+3.779,5213230.0,31.0,2.0,1:26.529,222.085,1
5,1,HAM,Lewis,Hamilton,British,91.0,23.0,38.0,1.0,1,...,1,10.0,76,2:00:42.742,7242742.0,71.0,6.0,1:18.510,153.152,1
6,1,HAM,Lewis,Hamilton,British,113.0,24.0,38.0,2.0,2,...,19,0.0,19,NaN,NaN,4.0,3.0,1:17.506,202.559,4
7,1,HAM,Lewis,Hamilton,British,135.0,25.0,38.0,4.0,4,...,10,0.0,70,+54.538,5564783.0,40.0,5.0,1:17.453,205.022,1


In [152]:
drivers_results_cleaning = (drivers_results_cols
            >> drop(X.position_y, X.position_x, X.positionOrder, X['rank'], X['number'], X['grid'], X.resultId, X.driverStandingsId, X.milliseconds)
) 
drivers_results_cleaning.head(10)

,driverId,code,fname,lname,nationality,raceId,running_season_total,season_position,wins,year,...,time_of_race,constructorId,race_position,race_points,laps,race_time,fastestLap,fastestLapTime,fastestLapSpeed,statusId
0,1,HAM,Lewis,Hamilton,British,18.0,10.0,1,1.0,2008,...,04:30:00,1,1,10.0,58,1:34:50.616,39.0,1:27.452,218.300,1
1,1,HAM,Lewis,Hamilton,British,19.0,14.0,1,1.0,2008,...,07:00:00,1,5,4.0,56,+46.548,53.0,1:35.462,209.033,1
2,1,HAM,Lewis,Hamilton,British,20.0,14.0,3,1.0,2008,...,11:30:00,1,13,0.0,56,NaN,25.0,1:35.520,203.969,11
3,1,HAM,Lewis,Hamilton,British,21.0,20.0,2,1.0,2008,...,12:00:00,1,3,6.0,66,+4.187,20.0,1:22.017,204.323,1
4,1,HAM,Lewis,Hamilton,British,22.0,28.0,3,1.0,2008,...,12:00:00,1,2,8.0,58,+3.779,31.0,1:26.529,222.085,1
5,1,HAM,Lewis,Hamilton,British,23.0,38.0,1,2.0,2008,...,12:00:00,1,1,10.0,76,2:00:42.742,71.0,1:18.510,153.152,1
6,1,HAM,Lewis,Hamilton,British,24.0,38.0,2,2.0,2008,...,17:00:00,1,R,0.0,19,NaN,4.0,1:17.506,202.559,4
7,1,HAM,Lewis,Hamilton,British,25.0,38.0,4,2.0,2008,...,12:00:00,1,10,0.0,70,+54.538,40.0,1:17.453,205.022,1
8,1,HAM,Lewis,Hamilton,British,26.0,48.0,1,3.0,2008,...,12:00:00,1,1,10.0,60,1:39:09.440,16.0,1:32.817,199.398,1
9,1,HAM,Lewis,Hamilton,British,27.0,58.0,1,4.0,2008,...,12:00:00,1,1,10.0,67,1:31:20.874,17.0,1:16.039,216.552,1


In [153]:
drivers_status = (drivers_results_cleaning >> left_join(status, by='statusId'))
drivers_status.head(5)

,driverId,code,fname,lname,nationality,raceId,running_season_total,season_position,wins,year,...,constructorId,race_position,race_points,laps,race_time,fastestLap,fastestLapTime,fastestLapSpeed,statusId,status
0,1,HAM,Lewis,Hamilton,British,18.0,10.0,1,1.0,2008,...,1,1,10.0,58,1:34:50.616,39.0,1:27.452,218.300,1,Finished
1,1,HAM,Lewis,Hamilton,British,19.0,14.0,1,1.0,2008,...,1,5,4.0,56,+46.548,53.0,1:35.462,209.033,1,Finished
2,1,HAM,Lewis,Hamilton,British,20.0,14.0,3,1.0,2008,...,1,13,0.0,56,NaN,25.0,1:35.520,203.969,11,+1 Lap
3,1,HAM,Lewis,Hamilton,British,21.0,20.0,2,1.0,2008,...,1,3,6.0,66,+4.187,20.0,1:22.017,204.323,1,Finished
4,1,HAM,Lewis,Hamilton,British,22.0,28.0,3,1.0,2008,...,1,2,8.0,58,+3.779,31.0,1:26.529,222.085,1,Finished


In [154]:
drivers_status.columns

Index(['driverId', 'code', 'fname', 'lname', 'nationality', 'raceId',
       'running_season_total', 'season_position', 'wins', 'year', 'race_round',
       'circuitId', 'circuit_name', 'time_of_race', 'constructorId',
       'race_position', 'race_points', 'laps', 'race_time', 'fastestLap',
       'fastestLapTime', 'fastestLapSpeed', 'statusId', 'status'],
      dtype='object')

In [155]:
personalmax = (drivers_status
        >> group_by(X.driverId, X.fname, X.lname, X.nationality, X.year)
        >> summarize(maxdrive = X.running_season_total.max())
)
personalmax.head(5)

,year,nationality,lname,fname,driverId,maxdrive
0,2007,British,Hamilton,Lewis,1,109.0
1,2008,British,Hamilton,Lewis,1,98.0
2,2009,British,Hamilton,Lewis,1,49.0
3,2010,British,Hamilton,Lewis,1,240.0
4,2011,British,Hamilton,Lewis,1,227.0


In [156]:
mostdriverpoint = (drivers_status
            >> group_by(X.year)
            >> summarize(drivermax = X.running_season_total.max())
)
mostdriverpoint.head(5)

,year,drivermax
0,1950,30.0
1,1951,31.0
2,1952,36.0
3,1953,34.5
4,1954,42.0


In [157]:
pd.set_option('display.max_rows', False) #None if max rows wanted

driver_max_join = (personalmax >> inner_join(mostdriverpoint,by='year'))
driver_championships = (driver_max_join
             >> mutate(driverpercent = X.maxdrive/X.drivermax)
             >> filter_by(X.driverpercent==1)
             >> arrange(X.year, ascending = True)
            #  >> drop(X.constructorId, X.team_percents, X.maxpoints)
)
driver_championships

,year,nationality,lname,fname,driverId,maxdrive,drivermax,driverpercent
2693,1950,Italian,Farina,Nino,642,30.0,30.0,1.0
2762,1951,Argentine,Fangio,Juan,579,31.0,31.0,1.0
2869,1952,Italian,Ascari,Alberto,647,36.0,36.0,1.0
2989,1953,Italian,Ascari,Alberto,647,34.5,34.5,1.0
3076,1954,Argentine,Fangio,Juan,579,42.0,42.0,1.0
2618,1955,Argentine,Fangio,Juan,579,40.0,40.0,1.0
2466,1956,Argentine,Fangio,Juan,579,30.0,30.0,1.0
2558,1957,Argentine,Fangio,Juan,579,40.0,40.0,1.0
2095,1958,British,Hawthorn,Mike,578,42.0,42.0,1.0
...,...,...,...,...,...,...,...,...


In [158]:
driver_count_championships = (driver_championships
                    >> group_by(X.fname, X.lname, X.driverId)
                    >> summarize(championships = X.driverId.count())
                    >> ungroup
                    >> arrange(X.championships, ascending = False)
                    >> filter_by(X.championships > 1)
)
driver_count_championships

,driverId,lname,fname,championships
24,30,Schumacher,Michael,7
21,1,Hamilton,Lewis,7
18,579,Fangio,Juan,5
0,117,Prost,Alain,4
33,20,Vettel,Sebastian,4
3,102,Senna,Ayrton,3
9,356,Brabham,Jack,3
10,328,Stewart,Jackie,3
27,137,Piquet,Nelson,3
30,182,Lauda,Niki,3


In [159]:
bestdrivers = (driver_count_championships
    >> filter_by(X.championships > 3)
)
bestdrivers

,driverId,lname,fname,championships
24,30,Schumacher,Michael,7
21,1,Hamilton,Lewis,7
18,579,Fangio,Juan,5
0,117,Prost,Alain,4
33,20,Vettel,Sebastian,4


In [160]:
remove_decimal = lambda x: x.replace('.0', '')

In [161]:
topdriver_select = (drivers_status
                >> mutate(driverId = X.driverId.astype(str))
                >> mutate(raceId = X.raceId.astype(str).apply(remove_decimal))
                # >> filter_by((X.driverId == '30')|(X.driverId == '1')|(X.driverId=='579')|(X.driverId=='579')|(X.driverId=='117')|(X.driverId=='20'))
)
topdriver_select.head(10)

,driverId,code,fname,lname,nationality,raceId,running_season_total,season_position,wins,year,...,constructorId,race_position,race_points,laps,race_time,fastestLap,fastestLapTime,fastestLapSpeed,statusId,status
0,1,HAM,Lewis,Hamilton,British,18,10.0,1,1.0,2008,...,1,1,10.0,58,1:34:50.616,39.0,1:27.452,218.300,1,Finished
1,1,HAM,Lewis,Hamilton,British,19,14.0,1,1.0,2008,...,1,5,4.0,56,+46.548,53.0,1:35.462,209.033,1,Finished
2,1,HAM,Lewis,Hamilton,British,20,14.0,3,1.0,2008,...,1,13,0.0,56,NaN,25.0,1:35.520,203.969,11,+1 Lap
3,1,HAM,Lewis,Hamilton,British,21,20.0,2,1.0,2008,...,1,3,6.0,66,+4.187,20.0,1:22.017,204.323,1,Finished
4,1,HAM,Lewis,Hamilton,British,22,28.0,3,1.0,2008,...,1,2,8.0,58,+3.779,31.0,1:26.529,222.085,1,Finished
5,1,HAM,Lewis,Hamilton,British,23,38.0,1,2.0,2008,...,1,1,10.0,76,2:00:42.742,71.0,1:18.510,153.152,1,Finished
6,1,HAM,Lewis,Hamilton,British,24,38.0,2,2.0,2008,...,1,R,0.0,19,NaN,4.0,1:17.506,202.559,4,Collision
7,1,HAM,Lewis,Hamilton,British,25,38.0,4,2.0,2008,...,1,10,0.0,70,+54.538,40.0,1:17.453,205.022,1,Finished
8,1,HAM,Lewis,Hamilton,British,26,48.0,1,3.0,2008,...,1,1,10.0,60,1:39:09.440,16.0,1:32.817,199.398,1,Finished
9,1,HAM,Lewis,Hamilton,British,27,58.0,1,4.0,2008,...,1,1,10.0,67,1:31:20.874,17.0,1:16.039,216.552,1,Finished


In [162]:
investigate = (drivers_status
        >> group_by(X.status)
        >> filter_by((X.race_position != "R") & (X.statusId != 1)) #this one is weird the cases where the status is everything but 1 (finished) there is an R in
)
investigate.head(10)

,driverId,code,fname,lname,nationality,raceId,running_season_total,season_position,wins,year,...,constructorId,race_position,race_points,laps,race_time,fastestLap,fastestLapTime,fastestLapSpeed,statusId,status
2,1,HAM,Lewis,Hamilton,British,20.0,14.0,3,1.0,2008,...,1,13,0.0,56,NaN,25.0,1:35.520,203.969,11,+1 Lap
28,1,HAM,Lewis,Hamilton,British,45.0,70.0,1,2.0,2007,...,1,9,0.0,59,NaN,29.0,1:33.401,198.421,11,+1 Lap
35,1,HAM,Lewis,Hamilton,British,52.0,109.0,2,4.0,2007,...,1,7,2.0,70,NaN,58.0,1:12.506,213.946,11,+1 Lap
39,1,HAM,Lewis,Hamilton,British,5.0,9.0,7,0.0,2009,...,1,9,0.0,65,NaN,29.0,1:23.839,199.883,11,+1 Lap
40,1,HAM,Lewis,Hamilton,British,6.0,9.0,9,0.0,2009,...,1,12,0.0,77,NaN,65.0,1:15.706,158.824,11,+1 Lap
42,1,HAM,Lewis,Hamilton,British,8.0,9.0,11,0.0,2009,...,1,16,0.0,59,NaN,24.0,1:22.576,224.128,11,+1 Lap
43,1,HAM,Lewis,Hamilton,British,9.0,9.0,11,0.0,2009,...,1,18,0.0,59,NaN,46.0,1:35.367,194.331,11,+1 Lap
114,1,HAM,Lewis,Hamilton,British,884.0,50.0,4,0.0,2013,...,131,12,0.0,65,NaN,53.0,1:27.895,190.659,11,+1 Lap
204,1,HAM,Lewis,Hamilton,British,986.0,333.0,1,9.0,2017,...,131,9,2.0,70,NaN,69.0,1:19.945,193.813,11,+1 Lap
290,1,HAM,Lewis,Hamilton,British,1077.0,28.0,7,0.0,2022,...,131,13,0.0,62,NaN,50.0,1:21.419,217.054,11,+1 Lap


In [163]:
positionselect = (topdriver_select
            >> select(X.raceId, X.driverId, X.race_position, X.race_round, X.circuitId, X.year)
            >> filter_by(text_filter(X.race_position, '[^A-Z]', regex=True))
            >> mutate(race_position = X.race_position.astype(int))
            >> mutate(raceId = X.raceId.astype(str).apply(remove_decimal))
)
positionselect.head(10)

,raceId,driverId,race_position,race_round,circuitId,year
0,18,1,1,1.0,1.0,2008
1,19,1,5,2.0,2.0,2008
2,20,1,13,3.0,3.0,2008
3,21,1,3,4.0,4.0,2008
4,22,1,2,5.0,5.0,2008
5,23,1,1,6.0,6.0,2008
7,25,1,10,8.0,8.0,2008
8,26,1,1,9.0,9.0,2008
9,27,1,1,10.0,10.0,2008
10,28,1,5,11.0,11.0,2008


In [164]:
positionselect.dtypes

raceId            object
driverId          object
race_position      int64
race_round       float64
circuitId        float64
year              object
dtype: object

In [165]:
statusrecode = {'Finished':'finished', '+1 Lap':'extra laps 5 or less', 'Collision':'driver/circuit', 'Spun off':'driver/circuit', 'Accident':'driver/circuit',
       'Brakes':'constructor_error', 'Puncture':'constructor_error', 'Gearbox':'constructor_error', 'Electrical':'constructor_error', 'Engine':'constructor_error', 
       'Retired':'other','Throttle':'constructor_error', 'Fuel pressure':'constructor_error', '+4 Laps':'extra laps 5 or less', 'Transmission':'constructor_error',
       'Technical':'constructor_error', '+2 Laps':'extra laps 5 or less', 'Withdrew':'Withdrew', 'Tyre':'constructor_error', 'Driveshaft':'constructor_error',
       'Hydraulics':'constructor_error', 'Handling':'constructor_error', 'Clutch':'constructor_error', '+3 Laps':'constructor_error', 'Alternator':'constructor_error',
       'Suspension':'constructor_error', 'Out of fuel':'constructor_error', 'Heat shield fire':'constructor_error', 'Water leak':'constructor_error',
       'Electronics':'constructor_error', 'Wheel':'constructor_error', 'ERS':'constructor_error', '+6 Laps':'extra laps 6 to 10', 'Battery':'constructor_error',
       'Vibrations':'constructor_error', 'Power Unit':'constructor_error', 'Collision damage':'driver_error', 'Exhaust':'constructor_error', 'Rear wing':'constructor_error',
       'Water pump':'constructor_error', '+10 Laps':'extra laps 6 to 10', 'Steering':'driver_error', 'Oil pressure':'constructor_error', 'Pneumatics':'constructor_error',
       'Wheel rim':'constructor_error', '+5 Laps':'extra laps 5 or less', 'Halfshaft':'constructor_error', 'Differential':'constructor_error', 'Brake duct':'constructor_error',
       'Water pressure':'constructor_error', 'Turbo':'driver_error', 'Power loss':'constructor_error', 'Overheating':'constructor_error', 'Oil leak':'constructor_error',
       'Driver Seat':'constructor_error', 'Injured':'driver_error', 'Track rod':'constructor_error', 'Mechanical':'constructor_error', '+9 Laps':'extra laps 6 to 10',
       '+17 Laps':'extra laps 16+', 'Disqualified':'DQ', '+7 Laps':'extra laps 6 to 10', 'Excluded':'driver_error', 'Wheel nut':'constructor_error',
       '+12 Laps':'extra laps 11 to 15', '+26 Laps':'extra laps 16+', 'Safety':'other', 'Fuel':'constructor_error', 'Radiator':'constructor_error',
       'Spark plugs':'constructor_error', '+8 Laps':'extra laps 6 to 10', 'Fuel system':'constructor_error', 'Injury':'driver_error', 'Fuel pump':'constructor_error',
       'Ignition':'constructor_error', 'Wheel bearing':'constructor_error', '+11 Laps':'extra laps 11 to 15', 'Front wing':'constructor_error',
       'Refuelling':'constructor_error', 'Broken wing':'constructor_error', 'Chassis':'constructor_error', 'Engine fire':'constructor_error',
       'Not restarted':'driver_error', 'Did not qualify':'driver_error', '+42 Laps':'extra laps 16+', 'Launch control':'constructor_error',
       'Tyre puncture':'constructor_error', 'Fire':'Misc.', 'Oil line':'constructor_error', 'Fuel rig':'constructor_error', 'Stalled':'constructor_error',
       '107% Rule':'constructor_error', 'Underweight':'constructor_error', 'Crankshaft':'constructor_error', 'Drivetrain':'constructor_error',
       'Not classified':'Misc.', 'Physical':'Misc.', 'Oil pump':'constructor_error', 'Injection':'constructor_error',
       'Did not prequalify':'driver_error', 'Fuel leak':'constructor_error', 'Driver unwell':'driver_error', 'CV joint':'constructor_error',
       'Distributor':'constructor_error', 'Fatal accident':'driver/circuit', 'Fuel pipe':'constructor_error', 'Oil pipe':'constructor_error',
       'Axle':'constructor_error',
       'Eye injury':'driver_error', '+25 Laps':'extra laps 16+', '+22 Laps':'extra laps 16+', 'Water pipe':'constructor_error', '+21 Laps':'extra laps 16+',
       '+24 Laps':'extra laps 16+', '+29 Laps':'extra laps 16+', '+14 Laps':'extra laps 11 to 15', '+15 Laps':'extra laps 11 to 15', '+16 Laps':'extra laps 16+',
       '+30 Laps':'extra laps 16+', '+23 Laps':'extra laps 16+', '+44 Laps':'extra laps 16+', '+18 Laps':'extra laps 16+', '+19 Laps':'extra laps 16+',
       '+13 Laps':'extra laps 11 to 15', 'Magneto':'Misc.', '+20 Laps':'extra laps 16+', '+46 Laps':'extra laps 16+', 'Supercharger':'constructor_error',
       'Engine misfire':'constructor_error', 'Seat':'Misc.', 'Damage':'constructor_error', 'Cooling system':'constructor_error', 'Illness':'driver_error',
       'Debris':'driver_error', 'Undertray':'constructor_error'}

do average but ALSO std dev to measure consistency of laps - look to see visually the speed with consistency

plot of avg lap time vs std dev lap time

logistic model/probability model of top 10 (points drop off after top 10)

In [166]:
def add_decimal(x):
    return float(str(x)[:-3] + "." + str(x)[-3:])

In [167]:
lapdrivers_select = (lap_times
        >> mutate(driverId = X.driverId.astype(str))
        >> mutate(raceId = X.raceId.astype(str))
        >> group_by(X.driverId, X.raceId)
        >> mutate(newtime = (X.milliseconds.apply(add_decimal)))
        >> select(~X.time, ~X.milliseconds)
        >> summarize(avg_laptime = X.newtime.mean(), 
                     lap_deviation = X.newtime.std())
        >> ungroup
)
lapdrivers_select.head(5)

,raceId,driverId,avg_laptime,lap_deviation
0,1,1,97.563759,16.066158
1,10,1,84.341086,4.220408
2,1000,1,83.377529,3.404940
3,1001,1,114.216750,18.541712
4,1002,1,87.065736,10.008775


In [168]:
## only has laptimes for lewis hamilton, michael schumacher, and sebastian vettel - lap times were not recorded for the previous drivers

# when comparing only need the following         
# # >> filter_by((X.driverId == '30')|(X.driverId == '1')|(X.driverId=='579')|(X.driverId=='117')|(X.driverId=='20'))


In [169]:
str_splitting = lambda x: list(map(float, x.split(':')))
min_to_seconds = (lambda x: (x[0]*60 + x[1]))

In [204]:
laptimes_joined = topdriver_select >> inner_join(lapdrivers_select, on =['driverId', 'raceId'])
driver_laptimes = (laptimes_joined
                >> select(X.driverId, X.fname, X.lname, X.raceId, X.year, X.laps, X.fastestLapTime, X.fastestLap, X.fastestLapSpeed, X.avg_laptime, X.lap_deviation, X.status)
                >> group_by(X.driverId, X.raceId)
                >> mutate(fastestLapTime = X.fastestLapTime.astype(str))
                >> filter_by(X.fastestLapTime != 'nan')
                >> mutate(fastestLapTime = X.fastestLapTime.apply(str_splitting))
                >> mutate(fastestLapTime = X.fastestLapTime.apply(min_to_seconds))
                >> mutate(status = X.status.map(statusrecode))
                >> mutate(new_statusId = if_else(X.status == 'driver/circuit', 1,0))
)
driver_laptimes.head(5)

,driverId,fname,lname,raceId,year,laps,fastestLapTime,fastestLap,fastestLapSpeed,avg_laptime,lap_deviation,status,new_statusId
44,1,Lewis,Hamilton,10,2009,70,82.479,16.0,191.219,84.341086,4.220408,finished,0
214,1,Lewis,Hamilton,1000,2018,70,81.107,63.0,194.454,83.377529,3.404940,finished,0
215,1,Lewis,Hamilton,1001,2018,44,106.721,23.0,236.264,114.216750,18.541712,finished,0
216,1,Lewis,Hamilton,1002,2018,53,82.497,30.0,252.794,87.065736,10.008775,finished,0
217,1,Lewis,Hamilton,1003,2018,61,102.913,56.0,177.108,109.370672,13.740921,finished,0


In [172]:
driver_laptimes.shape

(7155, 13)

1.111111e+61 is likely due to a precision error when adding large numbers.
This is happening because the x[0]*60 is being treated as a float, and it's too large for the float data type to handle, resulting in a precision loss.

The error message invalid literal for int() with base 10: '11111111111111111111111111111111111111111111111111111111111139.141' is indicating that the int() function is unable to convert the input value to an integer, because it is not a valid integer literal.
It looks like the problem is that the input value is a string and not a number, so the int() function is not able to convert it.


seems like it was trying to apply this function to a column that is not a list of numbers but a string.

time column is converted to a list of numbers using the list(map(float, x.split(':'))) function. The split(':') function is used to split the string by the colon character, and the map(float,...) function is used to convert the resulting list of strings to a list of numbers. Then, the function to convert the time in seconds is applied to the new list of numbers, and the time column is replaced by the time_in_seconds column and casted to float


In [173]:
driver_laptimes.dtypes

driverId            object
fname               object
lname               object
raceId              object
year                object
laps                 int64
fastestLapTime     float64
fastestLap         float64
fastestLapSpeed    float64
avg_laptime        float64
lap_deviation      float64
status              object
new_statusId         int64
dtype: object

In [174]:
pitstop_times =(pit_stops
        >> mutate(driverId = X.driverId.astype(str))
        >> mutate(raceId = X.raceId.astype(str))
        # >> filter_by((X.driverId == '30')|(X.driverId == '1')|(X.driverId=='579')|(X.driverId=='117')|(X.driverId=='20'))
        >> group_by(X.driverId, X.raceId)
        >> mutate(newtime = (X.milliseconds.apply(add_decimal)))
        >> select(~X.time, ~X.milliseconds)
        >> summarize(avg_pitstop = X.newtime.mean(), 
                     pit_dev = X.newtime.std())
        >> ungroup
)
pitstop_times.head(5)

,raceId,driverId,avg_pitstop,pit_dev
0,1000,1,21.4800,NaN
1,1001,1,20.8735,2.123442
2,1002,1,23.7280,NaN
3,1003,1,28.9460,NaN
4,1004,1,29.5510,NaN


In Formula 1 racing, the qualifying round is a session held before the main race that determines the starting positions for the race. There are typically three segments to the qualifying round, known as Q1, Q2, and Q3.

During Q1, all drivers are on the track at the same time and have a certain amount of time to set their fastest lap. The slowest drivers are eliminated, and the remaining drivers move on to Q2.

In Q2, the process is repeated, with the slowest drivers being eliminated and the remaining drivers moving on to Q3.

Finally, in Q3, the top ten drivers compete for the pole position, or the first starting spot for the race. The driver who sets the fastest lap time during Q3 will start in the pole position, with the second-fastest driver starting in second place, and so on.

The qualifying round is important because the starting position can have a significant impact on the outcome of the race, as drivers who start at the front of the grid have a better chance of winning the race

In [175]:
qualify_clean = (qualifying
            >> mutate(raceId = X.raceId.astype(str))
            >> mutate(driverId = X.driverId.astype(str))
            >> mutate(starting_position = X.position)
            >> drop(X.position, X.number, X.qualifyId)
)
qualify_clean.head(5)

,raceId,driverId,constructorId,q1,q2,q3,starting_position
0,18,1,1,1:26.572,1:25.187,1:26.714,1
1,18,9,2,1:26.103,1:25.315,1:26.869,2
2,18,5,1,1:25.664,1:25.452,1:27.079,3
3,18,13,6,1:25.994,1:25.691,1:27.178,4
4,18,2,2,1:25.960,1:25.518,1:27.236,5


In [176]:
alltimesjoined = driver_laptimes >> left_join(pitstop_times, on = ['raceId', 'driverId'])
alltimes_positions = alltimesjoined >> left_join(positionselect, on = ['raceId', 'driverId'])
alltimes_qualifying = alltimes_positions >> outer_join(qualify_clean, on = ['raceId','driverId'])
alltimes_qualifying.head(5)

,driverId,fname,lname,raceId,year,laps,fastestLapTime,fastestLap,fastestLapSpeed,avg_laptime,...,avg_pitstop,pit_dev,race_position,race_round,circuitId,constructorId,q1,q2,q3,starting_position
0,1,Lewis,Hamilton,10,2009,70.0,82.479,16.0,191.219,84.341086,...,NaN,NaN,1.0,10.0,11.0,1.0,1:20.842,1:20.465,1:21.839,4.0
1,1,Lewis,Hamilton,1000,2018,70.0,81.107,63.0,194.454,83.377529,...,21.4800,NaN,1.0,12.0,11.0,131.0,1:17.419,1:31.242,1:35.658,1.0
2,1,Lewis,Hamilton,1001,2018,44.0,106.721,23.0,236.264,114.216750,...,20.8735,2.123442,2.0,13.0,13.0,131.0,1:42.977,1:41.553,1:58.179,1.0
3,1,Lewis,Hamilton,1002,2018,53.0,82.497,30.0,252.794,87.065736,...,23.7280,NaN,1.0,14.0,14.0,131.0,1:20.810,1:19.798,1:19.294,3.0
4,1,Lewis,Hamilton,1003,2018,61.0,102.913,56.0,177.108,109.370672,...,28.9460,NaN,1.0,15.0,15.0,131.0,1:39.403,1:37.344,1:36.015,1.0


In [177]:
alltimes_qualifying.columns

Index(['driverId', 'fname', 'lname', 'raceId', 'year', 'laps',
       'fastestLapTime', 'fastestLap', 'fastestLapSpeed', 'avg_laptime',
       'lap_deviation', 'status', 'new_statusId', 'avg_pitstop', 'pit_dev',
       'race_position', 'race_round', 'circuitId', 'constructorId', 'q1', 'q2',
       'q3', 'starting_position'],
      dtype='object')

In [178]:
alltimes_qualifying.year.unique()

array(['2009', '2018', '2019', '2020', '2021', '2022', '2008', '2010',
       '2007', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2004', '2006', '2005', nan], dtype=object)

In [179]:
# alltimes_qualifying.to_csv('./data/alltimes_qualifying.csv')

for the following code only years have data only in 2004 and on up :/ - some drivers lap time, pit stop time, and qualifier information wasn't recorded until late 2000's/early 2010's

In [180]:
driver_thing = (drivers
            >> select(X.driverId, X.forename, X.surname)
            >> mutate(driverName = X.forename + ' ' + X.surname)
            >> drop(X.forename, X.surname)
)

lap_thing = (results
            >> select(X.raceId, X.driverId, X.positionOrder)
            >> mutate(finish_position = X.positionOrder)
            >> drop(X.positionOrder)
)

lap_drive_join = driver_thing >> inner_join(lap_thing, on = 'driverId')
lap_drive_join

,driverId,driverName,raceId,finish_position
0,1,Lewis Hamilton,18,1
1,1,Lewis Hamilton,19,5
2,1,Lewis Hamilton,20,13
3,1,Lewis Hamilton,21,3
4,1,Lewis Hamilton,22,2
5,1,Lewis Hamilton,23,1
6,1,Lewis Hamilton,24,19
7,1,Lewis Hamilton,25,10
8,1,Lewis Hamilton,26,1
...,...,...,...,...


In [181]:
races_thing = (races
        >> select(X.raceId, X.year)
)
qualifying_thing = (qualifying
            >> select(X.raceId, X.position, X.driverId)
            >> mutate(start_position = X.position)
            >> drop(X.position)
)

race_qual_join = races_thing >> inner_join(qualifying_thing, on = 'raceId')
race_qual_join

,raceId,year,driverId,start_position
0,1,2009,18,1
1,1,2009,22,2
2,1,2009,20,3
3,1,2009,9,4
4,1,2009,3,5
5,1,2009,10,6
6,1,2009,13,7
7,1,2009,15,8
8,1,2009,8,9
...,...,...,...,...


In [182]:
races_quals_join = race_qual_join >> inner_join(lap_drive_join, on = ['raceId', 'driverId'])
drivers_to_1994 = (races_quals_join 
                   >> filter_by(X.year < 2004)
                   >> group_by(X.year, X.driverId, X.driverName)
                   >> summarize(avg_starting_position = mean(X.start_position),
                                avg_finish_position = mean(X.finish_position))
                   >> ungroup
                   >> mutate(year = X.year.astype(str))
                   >> mutate(driverId = X.driverId.astype(str))
)
drivers_to_1994.head(5)

,driverName,driverId,year,avg_starting_position,avg_finish_position
0,David Coulthard,14,1994,5.625000,8.375000
1,Rubens Barrichello,22,1994,9.333333,12.066667
2,Michael Schumacher,30,1994,1.846154,5.615385
3,Olivier Panis,44,1994,15.133333,10.000000
4,Heinz-Harald Frentzen,49,1994,8.733333,14.000000


In [183]:
drivers_to_1994.dtypes

driverName                object
driverId                  object
year                      object
avg_starting_position    float64
avg_finish_position      float64
dtype: object

In [184]:
driverId_string = (driver_championships
                >> mutate(driverId = X.driverId.astype(str))
)
driverId_string.head(5)

,year,nationality,lname,fname,driverId,maxdrive,drivermax,driverpercent
2693,1950,Italian,Farina,Nino,642,30.0,30.0,1.0
2762,1951,Argentine,Fangio,Juan,579,31.0,31.0,1.0
2869,1952,Italian,Ascari,Alberto,647,36.0,36.0,1.0
2989,1953,Italian,Ascari,Alberto,647,34.5,34.5,1.0
3076,1954,Argentine,Fangio,Juan,579,42.0,42.0,1.0


In [185]:
driver_avg_pos = (alltimes_qualifying
                >> select(X.driverId, X.year, X.constructorId, X.starting_position, X.race_position)
                >> group_by(X.year, X.driverId, X.constructorId)
                >> summarize(avg_starting_position = mean(X.starting_position),
                             avg_finish_position = mean(X.race_position))
                >> ungroup
                >> mutate(constructorId = X.constructorId.astype(str))
                >> mutate(constructorId = X.constructorId.apply(remove_decimal))
)
driver_avg_pos.head(5)

,constructorId,driverId,year,avg_starting_position,avg_finish_position
0,17,10,2004,17.000000,13.000000
1,16,11,2004,7.294118,7.538462
2,15,13,2004,13.687500,9.214286
3,1,14,2004,9.555556,7.714286
4,4,15,2004,6.928571,5.583333


In [186]:
position_champion_join = driverId_string >> inner_join(driver_avg_pos, on = ['year', 'driverId'])
# constructor_with_driver = position_champion_join >> inner_join(constructor_champions, on = ['constructorId', 'year'])
# constructor_with_driver
position_champion_join.head(5)

,year,nationality,lname,fname,driverId,maxdrive,drivermax,driverpercent,constructorId,avg_starting_position,avg_finish_position
0,2004,German,Schumacher,Michael,30,148.0,148.0,1.0,6,3.444444,2.117647
1,2005,Spanish,Alonso,Fernando,4,133.0,133.0,1.0,4,3.944444,2.411765
2,2006,Spanish,Alonso,Fernando,4,134.0,134.0,1.0,4,4.333333,1.937500
3,2007,Finnish,Räikkönen,Kimi,8,110.0,110.0,1.0,6,3.588235,2.600000
4,2008,British,Hamilton,Lewis,1,98.0,98.0,1.0,1,3.055556,4.411765


In [187]:
champion_avg_position = (position_champion_join
                    >> select(X.year, X.fname, X.lname, X.driverId, X.avg_starting_position, X.constructorId, X.avg_finish_position)
                    >> mutate(driverName = X.fname + ' ' + X.lname)
                    >> select(X.year, X.driverId, X.constructorId, X.driverName, X.avg_starting_position, X.avg_finish_position)
)
champion_avg_position.head(5)

,year,driverId,constructorId,driverName,avg_starting_position,avg_finish_position
0,2004,30,6,Michael Schumacher,3.444444,2.117647
1,2005,4,4,Fernando Alonso,3.944444,2.411765
2,2006,4,4,Fernando Alonso,4.333333,1.937500
3,2007,8,6,Kimi Räikkönen,3.588235,2.600000
4,2008,1,1,Lewis Hamilton,3.055556,4.411765


In [188]:
# champion_avg_position.to_csv('./data/champion_avg_position.csv')

circuit ranking by charactersics/status - which data characterstics will indicate whether or not the circuit is more difficult or not (accidents, variation in speed, driver winner consistency)

potentially cluster circuits together

In [189]:
status_with_drivers = (drivers_status
        >> mutate(status = X.status.map(statusrecode))
        >> mutate(new_statusId = if_else(X.status == 'driver/circuit', 1,0))
        # >> filter_by(X.new_statusId == 1)
)

status_with_drivers.head(5)

,driverId,code,fname,lname,nationality,raceId,running_season_total,season_position,wins,year,...,race_position,race_points,laps,race_time,fastestLap,fastestLapTime,fastestLapSpeed,statusId,status,new_statusId
0,1,HAM,Lewis,Hamilton,British,18.0,10.0,1,1.0,2008,...,1,10.0,58,1:34:50.616,39.0,1:27.452,218.300,1,finished,0
1,1,HAM,Lewis,Hamilton,British,19.0,14.0,1,1.0,2008,...,5,4.0,56,+46.548,53.0,1:35.462,209.033,1,finished,0
2,1,HAM,Lewis,Hamilton,British,20.0,14.0,3,1.0,2008,...,13,0.0,56,NaN,25.0,1:35.520,203.969,11,extra laps 5 or less,0
3,1,HAM,Lewis,Hamilton,British,21.0,20.0,2,1.0,2008,...,3,6.0,66,+4.187,20.0,1:22.017,204.323,1,finished,0
4,1,HAM,Lewis,Hamilton,British,22.0,28.0,3,1.0,2008,...,2,8.0,58,+3.779,31.0,1:26.529,222.085,1,finished,0


In [190]:
status_with_drivers.shape

(25191, 25)

In [249]:
circjoin = status_with_drivers >> inner_join(circuits, by='circuitId')
circ_cleaning = (circjoin
            >> drop(X.url, X.lat, X.lng, X['name'])
            >> mutate(driverId = X.driverId.astype(str))
            >> mutate(raceId = X.raceId.astype(str))
            >> mutate(raceId = X.raceId.map(remove_decimal))
            >> mutate(circuitId = X.circuitId.astype(str))
            >> mutate(circuitId = X.circuitId.map(remove_decimal))
)
# circ_cleaning.head(5)
circuit_laptimes = circ_cleaning >> inner_join(driver_laptimes, by = ['driverId', 'raceId'])
circuit_laptimes_cleaning = (circuit_laptimes
                        >> select(X.circuitRef, X.circuitId, X.driverId, X.code, X.raceId, X.alt, X.laps_x, X.year_y, X.fastestLapTime_y,
                                  X.fastestLap_y, X.fastestLapSpeed_y, X.avg_laptime, X.lap_deviation, X.status_y, X.new_statusId_y)
)


circuit_laptimes_columns = (circuit_laptimes_cleaning.rename(columns={"circuitRef":"circuitRef", "driverId":"driverId", "code":"driverCode", "raceId":"raceId",
                                                                   "time_of_race":"time_of_race", "alt":"altitude", "laps_x":"laps", "laps_y":"laps_y",
                                                                   "fastestLapTime_y":"fastestLapTime", "fastestLapSpeed_y":"fastestLapSpeed", "avg_laptime":"avg_laptime",
                                                                   "lap_deviation":"lap_deviation", "status_y":"status", "new_statusId_y":"new_statusId", 
                                                                   "fastestLap_y":"fastestLap", "year_y":"year"}))

circuit_laptimes_columns.head(5)

# circuits_filter = (circuit_laptimes_columns
#                 >> filter_by(X.new_statusId == 1)
# )
# circuits_filter

,circuitRef,circuitId,driverId,driverCode,raceId,altitude,laps,year,fastestLapTime,fastestLap,fastestLapSpeed,avg_laptime,lap_deviation,status,new_statusId
0,albert_park,1,1,HAM,18,10.0,58,2008,87.452,39.0,218.300,98.114069,20.990357,finished,0
1,albert_park,1,1,HAM,969,10.0,57,2017,87.033,44.0,219.351,88.800825,2.724904,finished,0
2,albert_park,1,1,HAM,36,10.0,58,2007,86.351,20.0,221.083,88.747672,4.088674,finished,0
3,albert_park,1,1,HAM,338,10.0,58,2010,88.506,47.0,215.700,97.352224,16.424708,finished,0
4,albert_park,1,1,HAM,841,10.0,58,2011,90.314,41.0,211.382,92.975103,4.264972,finished,0


In [250]:
circuit_laptimes_columns.shape

(7155, 15)

In [251]:
circuit_laptimes_columns.year.unique()

array(['2008', '2017', '2007', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2018', '2019', '2022', '2006', '2009', '2005',
       '2004', '2020', '2021'], dtype=object)

In [215]:
circuits_time_join = alltimes_qualifying >> inner_join(circuits, by = ['circuitId'])
circuits_time_clean = (circuits_time_join
                    >> drop(X.lat, X.lng, X['name'], X.race_round, X.lap_deviation, X.pit_dev, X.url, X.q1, X.q2, X.q3)
                    >> select(X.circuitRef, X.alt, X.raceId, X.driverId, X.year, X.laps, X.fastestLapTime, X.fastestLap, X.fastestLapSpeed, X.avg_laptime)

)
circuits_time_clean.head(5)

,circuitRef,alt,raceId,driverId,year,laps,fastestLapTime,fastestLap,fastestLapSpeed,avg_laptime
0,hungaroring,264.0,10,1,2009,70.0,82.479,16.0,191.219,84.341086
1,hungaroring,264.0,1000,1,2018,70.0,81.107,63.0,194.454,83.377529
2,hungaroring,264.0,1021,1,2019,70.0,78.528,60.0,200.840,81.482800
3,hungaroring,264.0,1033,1,2020,70.0,76.627,70.0,205.823,82.463900
4,hungaroring,264.0,1062,1,2021,70.0,78.715,49.0,200.363,106.941929


In [216]:
circuits_time_clean.shape

(6066, 10)